# KYB AI Engine Evals

This notebook analyzes the data ouput by the KYB AI Engine. 

* Key Concepts from Compliance *
The Compliance teams 

## Load Data

In [1]:
import pandas as pd
import numpy as np
import os
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

load_dotenv()

conn_params = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT", "5432"),
}

### Querying PostgreSQL Database

In [ ]:
# Template function for fetching PostgreSQL tables as DataFrames
def fetch_table_as_dataframe(table_name, where_clause=None, params=None):
    """
    Fetch an entire PostgreSQL table and convert it to a pandas DataFrame.
    
    Args:
        table_name (str): Name of the table to fetch
        where_clause (str, optional): WHERE clause for filtering (without 'WHERE' keyword)
        params (tuple, optional): Parameters for the WHERE clause
    
    Returns:
        pd.DataFrame: The table data as a DataFrame
    """
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            # Build the query
            if where_clause:
                query = f"SELECT * FROM {table_name} WHERE {where_clause}"
            else:
                query = f"SELECT * FROM {table_name}"
            
            # Execute query
            if params:
                cur.execute(query, params)
            else:
                cur.execute(query)
            
            # Fetch all rows and column names
            rows = cur.fetchall()
            columns = [desc[0] for desc in cur.description]
    
    # Create DataFrame
    df = pd.DataFrame(rows, columns=columns)
    print(f"Table '{table_name}': {len(df)} rows fetched")
    return df

# Example usage for multiple tables
print("=== Fetching multiple tables ===")

# 1. Fetch entire kyb_job table
kyb_job_df = fetch_table_as_dataframe("kyb_job")
print(f"kyb_job table shape: {kyb_job_df.shape}")
print("\nFirst few rows of kyb_job:")
print(kyb_job_df.head())

# 2. Fetch entire file table
file_df = fetch_table_as_dataframe("file")
print(f"\nfile table shape: {file_df.shape}")
print("\nFirst few rows of file:")
print(file_df.head())

# 3. Fetch file table with WHERE clause (example)
if len(kyb_job_df) > 0:
    creator_id = kyb_job_df.iloc[0, 0]  # Get first creator_id from kyb_job
    file_filtered_df = fetch_table_as_dataframe("file", "creator_id = %s", (creator_id,))
    print(f"\nFiltered file table for creator_id {creator_id}: {file_filtered_df.shape}")
    print("\nFirst few rows of filtered file:")
    print(file_filtered_df.head())

NameError: name 'entity_id' is not defined

In [ ]:
# Additional examples and best practices for multiple table queries

# Example 1: Fetch multiple tables at once
def fetch_multiple_tables(table_names):
    """
    Fetch multiple tables and return them as a dictionary of DataFrames.
    
    Args:
        table_names (list): List of table names to fetch
    
    Returns:
        dict: Dictionary with table names as keys and DataFrames as values
    """
    tables = {}
    for table_name in table_names:
        tables[table_name] = fetch_table_as_dataframe(table_name)
    return tables

# Example 2: Fetch tables with custom queries
def fetch_custom_query(query, params=None):
    """
    Execute a custom SQL query and return results as DataFrame.
    
    Args:
        query (str): SQL query to execute
        params (tuple, optional): Parameters for the query
    
    Returns:
        pd.DataFrame: Query results as DataFrame
    """
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            if params:
                cur.execute(query, params)
            else:
                cur.execute(query)
            
            rows = cur.fetchall()
            columns = [desc[0] for desc in cur.description]
    
    df = pd.DataFrame(rows, columns=columns)
    print(f"Custom query returned: {len(df)} rows")
    return df

# Example 3: Fetch tables with joins
def fetch_joined_tables():
    """
    Example of fetching data from multiple tables using JOINs.
    """
    join_query = """
    SELECT k.*, f.* 
    FROM kyb_job k 
    LEFT JOIN file f ON k.id = f.creator_id
    """
    
    joined_df = fetch_custom_query(join_query)
    print(f"Joined table shape: {joined_df.shape}")
    return joined_df

# Example usage:
print("\n=== Additional Examples ===")

# Fetch multiple tables at once
print("\n1. Fetching multiple tables:")
table_names = ["kyb_job", "file"]  # Add more table names as needed
all_tables = fetch_multiple_tables(table_names)

# Display info about each table
for table_name, df in all_tables.items():
    print(f"{table_name}: {df.shape[0]} rows, {df.shape[1]} columns")
    print(f"Columns: {list(df.columns)}")
    print()

# Example of custom query
print("\n2. Custom query example:")
custom_df = fetch_custom_query("SELECT COUNT(*) as total_jobs FROM kyb_job")
print(custom_df)

# Example of joined tables (uncomment if you want to test)
# print("\n3. Joined tables example:")
# joined_df = fetch_joined_tables()
# print(joined_df.head())


## Implementation of Query

In [7]:
import pandas as pd
import numpy as np
import os
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

load_dotenv()

conn_params = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT", "5432"),
}

def fetch_table_as_dataframe(kyb_job, where_clause=None, params=None):
    """
    Fetch an entire PostgreSQL table and convert it to a pandas DataFrame.
    
    Args:
        table_name (str): Name of the table to fetch
        where_clause (str, optional): WHERE clause for filtering (without 'WHERE' keyword)
        params (tuple, optional): Parameters for the WHERE clause
    
    Returns:
        pd.DataFrame: The table data as a DataFrame
    """
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            # Build the query
            if where_clause:
                query = f"SELECT * FROM {table_name} WHERE {where_clause}"
            else:
                query = f"SELECT * FROM {table_name}"
            
            # Execute query
            if params:
                cur.execute(query, params)
            else:
                cur.execute(query)
            
            # Fetch all rows and column names
            rows = cur.fetchall()
            columns = [desc[0] for desc in cur.description]
    
    # Create DataFrame
    kyb_job_df = pd.DataFrame(rows, columns=columns)
    print(f"Table '{table_name}': {len(kyb_job_df)} rows fetched")
    return kyb_job_df